In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
import getpass
import pdvega
import time
from configobj import ConfigObj
from glob import glob

In [2]:
# Create a posgresql database connection using settings from config file 

In [11]:
df1=pd.read_csv(os.getcwd()+'\\diagnosis_com.csv')
df2=pd.read_csv(os.getcwd()+'\\intakeoutput_com.csv')
df3=pd.read_csv(os.getcwd()+'\\lab_com.csv')
df4=pd.read_csv(os.getcwd()+'\\microlab_com.csv')
df5=pd.read_csv(os.getcwd()+'\\respiratorycharting_com.csv')
df6=pd.read_csv(os.getcwd()+'\\vitalperiodicexe.csv')
df7=pd.read_csv(os.getcwd()+'\\allpatientfile.csv')
df8=pd.read_csv(os.getcwd()+'\\infusiondrug.csv')
df9=pd.read_csv(os.getcwd()+'\\dialysisoffset.csv')
df10=pd.read_csv(os.getcwd()+'\\dialysisday.csv')
df11=pd.read_csv(os.getcwd()+'\\respchartoffset.csv')

In [12]:
def main():
    ftName0=['patientunitstayid','gender','age','height','weight','Apachescore','ESRD']
    ftName1 = ['MI', 'CHF', 'PVD', 'CVD', 'Dementia', 'CPD', 'RD', 'PUD',
                    'MLD', 'DM_acute', 'DM_Chronic', 'Hemiplegia', 'Renal', 'Malignancy', 
                    'LD', 'MST', 'AIDS']
    ftName2 = ['intaketotal', 'outputtotal', 'dialysistotal', 'nettotal']
    ftName3 = ['platelets x 1000', 'WBC x 1000', 'Hgb', 'chloride', 'glucose', 'potassium', 'calcium', 'BUN', 
                    'creatinine', 'sodium', 'bedside glucose', 'magnesium', 'bicarbonate', 'total bilirubin', 'alkaline phos.',
                    'paCO2', 'paO2', 'HCO3', 'troponin - I', 'albumin', 'pH', 'total protein',
                    'phosphate', 'direct bilirubin', 'PT - INR',  'lactate', 'fibrinogen', 'CPK-MB INDEX', 'CPK',
                    'CPK-MB', 'HDL', 'LDL', 'total cholesterol',  'TSH', 'ammonia',
                    'amylase', 'lipase', 'T4', 'Vitamin B12', 'Fe', 'TIBC', 'ionized calcium', 'Ferritin',
                    'cortisol', 'free T4', 'T3', 'uric acid', 'serum osmolality', 'BNP', 'troponin - T',
                    'CRP', 'Fe/TIBC Ratio', 'LDH', 'transferrin', 'prealbumin', 'CRP-hs']
    ftName4 = ['Aspergillus', 'Candida', 'GNB', 'GPC']
    ftName5 = ['Compliance', 'FiO2', 'Mean airway pressure',
                    'Minute ventilation', 'PC mode', 'Peak airway pressure', 'PEEP',
                    'Plateau', 'Pressure support', 'Respiratory rate', 
                    'RSBI', 'tidal volume', 'ROXindex']
    ftName6 = ['sao2', 'heartrate',
               'systemicsystolic','systemicdiastolic','systemicmean']
    ftName8 = ['Vasopressor', 'Relaxant', 'Sedation', 'PPI']
    
    for k in range(0,len(ftName1)):
        for day in range(1,3):
            ftName0.append('%s D%02d'%(ftName1[k],day))
    for k in range(0,len(ftName2)):
        for day in range(1,3):
            ftName0.append('%s D%02d'%(ftName2[k],day))
    for k in range(0,len(ftName3)):
        for day in range(1,3):
            ftName0.append('%s D%02d'%(ftName3[k],day))
    for k in range(0,len(ftName4)):
        for day in range(1,3):
            ftName0.append('%s D%02d'%(ftName4[k],day))
    for k in range(0,len(ftName5)):
        for day in range(1,3):
            ftName0.append('%s D%02d'%(ftName5[k],day))
    for k in range(0,len(ftName8)):
        for day in range(1,3):
            ftName0.append('%s D%02d'%(ftName8[k],day))
    for k in range(0,len(ftName6)):
        for times in range(0,576):
            ftName0.append('%s D01-%02d'%(ftName6[k],times+1))
    ftName0.append('target')
    ftName0.append('Ventilator-day')
    ftName0.append('targetday=dialysisday')
    ftName0.append('dialysisday')
    ftName0.append('D3 having dialysis')
    pidoffset=[]
    pidday=[]
    for i in range(0,len(df9)):
        pidoffset.append(df9['patientunitstayid'][i])
    for i in range(0,len(df10)):
        pidday.append(df10['patientunitstayid'][i])
    df0=pd.DataFrame(columns=ftName0)
    count=0
    start=0
    end=0
    space_row = []
    for a in range(0,len(ftName0)):
        space_row.append(0)
    for i in range(0,len(df7)):
        j=0
        if(df7['patientunitstayid'][i] in pidoffset and df7['patientunitstayid'][i] in pidday and not pd.isna(df10['dialysisday %02d'%(j)][pidday.index(df7['patientunitstayid'][i])])): 
            for j in range(0,30):#dialysis day count
                if(not pd.isna(df10['dialysisday %02d'%(j)][pidday.index(df7['patientunitstayid'][i])])
                   and df5['target D%02d'%(df10['dialysisday %02d'%(j)][pidday.index(df7['patientunitstayid'][i])])][i]==1):
                    
                    df0 = df0.append(pd.Series(space_row, index=df0.columns), ignore_index=True)
                    df0['patientunitstayid'][count]=df7['patientunitstayid'][i]
                    df0['gender'][count]=df7['gender'][i]
                    df0['age'][count]=df7['age'][i]
                    df0['height'][count]=df7['height'][i]
                    df0['weight'][count]=df7['weight'][i]
                    df0['Apachescore'][count]=df7['Apachescore'][i]
                    df0['ESRD'][count]=df7['ESRD'][i]
                    for k in range(0,len(ftName1)):
                        for day in range(0,2):
                            df0['%s D%02d'%(ftName1[k],day+1)][count]=df1['%s D%02d'%(ftName1[k],df10['dialysisday %02d'%(j)][pidday.index(df7['patientunitstayid'][i])]+day)][i]
                    for k in range(0,len(ftName2)):
                        for day in range(0,2):
                            df0['%s D%02d'%(ftName2[k],day+1)][count]=df2['%s D%02d'%(ftName2[k],df10['dialysisday %02d'%(j)][pidday.index(df7['patientunitstayid'][i])]+day)][i]
                    for k in range(0,len(ftName3)):
                        for day in range(0,2):
                            df0['%s D%02d'%(ftName3[k],day+1)][count]=df3['%s D%02d'%(ftName3[k],df10['dialysisday %02d'%(j)][pidday.index(df7['patientunitstayid'][i])]+day)][i]
                    for k in range(0,len(ftName4)):
                        for day in range(0,2):
                            df0['%s D%02d'%(ftName4[k],day+1)][count]=df4['%s D%02d'%(ftName4[k],df10['dialysisday %02d'%(j)][pidday.index(df7['patientunitstayid'][i])]+day)][i]
                    for k in range(0,len(ftName5)):
                        for day in range(0,2):
                            df0['%s D%02d'%(ftName5[k],day+1)][count]=df5['%s D%02d'%(ftName5[k],df10['dialysisday %02d'%(j)][pidday.index(df7['patientunitstayid'][i])]+day)][i]
                    for k in range(0,len(ftName8)):
                        for day in range(0,2):
                            df0['%s D%02d'%(ftName8[k],day+1)][count]=df8['%s D%02d'%(ftName8[k],df10['dialysisday %02d'%(j)][pidday.index(df7['patientunitstayid'][i])]+day)][i]
                            
                    df0['Ventilator-day'][count]=df11['respchart %02d'%j][pidday.index(df7['patientunitstayid'][i])]
                    if(df10['dialysisday %02d'%(j)][pidday.index(df7['patientunitstayid'][i])]==df7['LoS'][i] and df7['hospitaldischargestatus'][i]=='Expired'):
                        df0['target'][count]=1
                    if(df7['LoS'][i]-df10['dialysisday %02d'%(j)][pidday.index(df7['patientunitstayid'][i])]<=2 and df7['hospitaldischargestatus'][i]=='Expired'):
                        df0['target'][count]=1
                    else:
                        try:
                            df0['target'][count]=df5['%s D%02d'%('target',df10['dialysisday %02d'%(j)][pidday.index(df7['patientunitstayid'][i])]+3)][i]
                        except KeyError:
                            df0['target'][count]=df5['%s D%02d'%('target',df10['dialysisday %02d'%(j)][pidday.index(df7['patientunitstayid'][i])]+2)][i]
                    
                    for l in range(j,30):
                        if(df10['dialysisday %02d'%(l)][pidday.index(df7['patientunitstayid'][i])]==df10['dialysisday %02d'%(j)][pidday.index(df7['patientunitstayid'][i])]+3):
                            df0['targetday=dialysisday'][count]=1
                            break
                        if(pd.isna(df10['dialysisday %02d'%(l)][pidday.index(df7['patientunitstayid'][i])])):
                            df0['targetday=dialysisday'][count]=0
                            break
                    for l in range(j,30):
                        if(df10['dialysisday %02d'%(l)][pidday.index(df7['patientunitstayid'][i])]==df10['dialysisday %02d'%(j)][pidday.index(df7['patientunitstayid'][i])]+2):
                            df0['D3 having dialysis'][count]=1
                            break
                        if(pd.isna(df10['dialysisday %02d'%(l)][pidday.index(df7['patientunitstayid'][i])])):
                            df0['D3 having dialysis'][count]=0
                            break
                    df0['dialysisday'][count]=df10['dialysisday %02d'%(j)][pidday.index(df7['patientunitstayid'][i])]
                    count+=1
                    if(pd.isna(df10['dialysisday %02d'%(j+1)][pidday.index(df7['patientunitstayid'][i])])):
                        break
                        
                else:
                    continue
        else:
            continue
    
    count=0
    for i in range(0,len(df0)):
        for k in range(0,len(ftName6)):
            count2=0
            start=df9['dialysisstart'][i]
            end=df9['dialysisstart'][i]+576
            if(df9['dialysisend'][i]-df9['dialysisstart'][i]<575):
                end=df9['dialysisend'][i]
            for times in range(start,end):
                df0['%s D01-%02d'%(ftName6[k],count2+1)][count]=df6['%s'%(ftName6[k])][times]
                count2+=1
                if(times==end-1 and end-start<575):
                    for com in range(end,start+576):
                        df0['%s D01-%02d'%(ftName6[k],count2+1)][count]=df6['%s'%(ftName6[k])][times]
                        count2+=1
                
        count+=1
        
    df0.to_csv(os.getcwd()+'\\inputdata.csv',index=False)
    
            

In [13]:
if __name__ is '__main__':
    main()#create dialysis offset csv file 

In [14]:
df=pd.read_csv(os.getcwd()+'\\inputdata.csv')
for i in range(0,len(df)):
    if(df['target'][i]==0):
        df['target'][i]=1
    
    elif(df['target'][i]==1):
        df['target'][i]=0
df.to_csv(os.getcwd()+'\\inputdata.csv',index=False)

c:\users\userpc\miniconda3\envs\243\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
c:\users\userpc\miniconda3\envs\243\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
c:\users\userpc\miniconda3\envs\243\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [15]:
df=pd.read_csv(os.getcwd()+'\\inputdata.csv')
for i in range(0,len(df)):
    if(df['gender'][i]=='Male'):
        df['gender'][i]=1
    if(df['gender'][i]=='Female'):
        df['gender'][i]=0
    if(df['age'][i]=='> 89'):
        df['age'][i]=90
df.to_csv(os.getcwd()+'\\inputdata.csv',index=False)

c:\users\userpc\miniconda3\envs\243\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
c:\users\userpc\miniconda3\envs\243\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\users\userpc\miniconda3\envs\243\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [16]:
df=pd.read_csv(os.getcwd()+'\\inputdata.csv')
pid=[]
for i in range(0,len(df)):
    pid.append(df['patientunitstayid'][i])
temp=[]
for i in range(0,len(df)):
    count=0
    for j in range(0,len(pid)):
        if(pid[j]==df['patientunitstayid'][i]):
            count+=1
    temp.append(count)
df.insert(len(df.columns),"count",temp)
droplist=[]
for i in range(0,len(df)):
    if(df['count'][i]>=7):
        droplist.append(i)
        
df=df.drop(droplist)
df.to_csv(os.getcwd()+'\\inputdatatemp.csv',index=False)

In [17]:
df=pd.read_csv(os.getcwd()+'\\inputdatatemp.csv')
df2=pd.DataFrame(columns=df.columns)
space=[]
for i in range(0,len(df.columns)):
    space.append(-0.01)
pid=[]
for i in range(0,len(df)-1):
    pid.append(df['patientunitstayid'][i])
pid=np.unique(pid)
start=0
end=0
flag=len(df)
count=0
for i in range(0,len(pid)):
    print('%i/%i: %s\r'%(i,len(pid),pid[i]),end="")
    flag=0
    for j in range(0,len(df)-1):
        if(flag==0 and df['patientunitstayid'][j]==pid[i]):
            start=j
            flag=1
        if(flag==1 and df['patientunitstayid'][j+1]!=pid[i]):
            end=j+1
            flag=2
        if(flag==2):
            target=[]
            for k in range(start,end):
                target.append(df['target'][k])
            if(1 in target and 0 in target):
                if(target==[0,1]):
                    df2=df2.append(pd.Series(space, index=df2.columns), ignore_index=True)
                    for l in df.columns:
                        df2[l][count]=df[l][start+target.index(1)]
                    count+=1
                else:
                    df2=df2.append(pd.Series(space, index=df2.columns), ignore_index=True)
                    for l in df.columns:
                        df2[l][count]=df[l][start]
                    count+=1
                    df2=df2.append(pd.Series(space, index=df2.columns), ignore_index=True)
                    for l in df.columns:
                        df2[l][count]=df[l][start+target.index(1)]
                    count+=1
            else:
                for k in range(start,end):
                    df2=df2.append(pd.Series(space, index=df2.columns), ignore_index=True)
                    for l in df.columns:
                        df2[l][count]=df[l][k]
                    count+=1
            
                '''
                if(df['target'][end-1]==1 and df['target'][k]==0):
                    df2=df2.append(pd.Series(space, index=df2.columns), ignore_index=True)
                    for l in df.columns:
                        df2[l][count]=df[l][end-1]
                    count+=1
                if(df['target'][end-1]==0 and df['target'][k]==0):
                    df2=df2.append(pd.Series(space, index=df2.columns), ignore_index=True)
                    for l in df.columns:
                        df2[l][count]=df[l][k]
                    count+=1
                if(df['target'][end-1]==0 and k==end-1):
                    df2=df2.append(pd.Series(space, index=df2.columns), ignore_index=True)
                    for l in df.columns:
                        df2[l][count]=df[l][k]
                    count+=1
                    '''
            break
df2.to_csv(os.getcwd()+'\\inputdatatemp.csv',index=False)

In [19]:
import math
df=pd.read_csv(os.getcwd()+'\\inputdatatemp.csv')
df2=pd.read_csv(os.getcwd()+'\\dialysisoffset.csv')
df3=pd.read_csv(os.getcwd()+'\\vitalperiodicexe.csv')

ftname=['heartrate','sao2','systemicsystolic','systemicdiastolic']
for i in ftname:
    temp=[]
    temp_1=[]
    temp_2=[]
    temp_3=[]
    print(i)
    if(i=='heartrate'):
        for j in range(0,len(df3)):
            if(not pd.isna(df3[i][j])):
                temp.append(df3[i][j])
        mean=round(np.nanmean(temp))
        deviation=round(np.nanstd(temp))
        
        for j in range(0,len(df)):
            count_1=0
            count_2=0
            count_3=0
            total=0
            for k in range(df2['dialysisstart'][j],df2['dialysisend'][j]):
                if(mean-deviation<=df3['heartrate'][k]<=mean+deviation):
                    count_1+=1
                    total+=1
                else:
                    count_2+=1
                    total+=1
            temp_1.append(count_1)
            temp_2.append(count_2)
    elif(i=='sao2'):
        for j in range(0,len(df3)):
            if(not pd.isna(df3[i][j])):
                temp.append(df3[i][j])
        mean=round(np.nanmean(temp))
        deviation=round(np.nanstd(temp))
        #print(mean,std)
        for j in range(0,len(df)):
            count_1=0
            count_2=0
            count_3=0
            total=0
            for k in range(df2['dialysisstart'][j],df2['dialysisend'][j]):
                if(mean-deviation<=df3['sao2'][k]<=mean):
                    count_1+=1
                    total+=1
                else:
                    count_2+=1
                    total+=1
            temp_1.append(count_1)
            temp_2.append(count_2)
    elif(i=='systemicsystolic'):
        for j in range(0,len(df3)):
            if(not pd.isna(df3[i][j])):
                temp.append(df3[i][j])
        mean=round(np.nanmean(temp))
        devitaion=round(np.nanstd(temp))
        
        for j in range(0,len(df)):
            count_1=0
            count_2=0
            count_3=0
            total=0
            for k in range(df2['dialysisstart'][j],df2['dialysisend'][j]):
                if(mean-deviation<=df3['systemicsystolic'][k]<=mean):
                    count_1+=1
                    total+=1
                else:
                    count_2+=1
                    total+=1
            temp_1.append(count_1)
            temp_2.append(count_2)
    elif(i=='systemicdiastolic'):
        for j in range(0,len(df3)):
            if(not pd.isna(df3[i][j])):
                temp.append(df3[i][j])
        mean=round(np.nanmean(temp))
        deviation=round(np.nanstd(temp))
        for j in range(0,len(df)):
            count_1=0
            count_2=0
            count_3=0
            total=0
            for k in range(df2['dialysisstart'][j],df2['dialysisend'][j]):
                if(mean-deviation<=df3['systemicdiastolic'][k]<=mean):
                    count_1+=1
                    total+=1 
                else:
                    count_2+=1
                    total+=1
            temp_1.append(count_1)
            temp_2.append(count_2)
    df.insert(len(df.columns),"Number of %s situations are normal"%i,temp_1)
    df.insert(len(df.columns),"Number of %s situations are abnormal"%i,temp_2)
df.to_csv(os.getcwd()+'\\inputdatatemp.csv',index=False)

heartrate
sao2
systemicsystolic
systemicdiastolic


In [20]:
df=pd.read_csv(os.getcwd()+'\\inputdatatemp.csv')
for i in df.columns:
    print(i,end='\r')
    temp=[]
    for j in range(0,len(df)):
        if(not pd.isna(df[i][j])):
            temp.append(df[i][j])
    for j in range(0,len(df)):
        if(pd.isna(df[i][j])):
            df[i][j]=np.mean(temp)
df.to_csv(os.getcwd()+'\\inputdatatemp_inputation.csv',index=False)

Apachescore

c:\users\userpc\miniconda3\envs\243\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


c:\users\userpc\miniconda3\envs\243\lib\site-packages\numpy\core\fromnumeric.py:3420: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
c:\users\userpc\miniconda3\envs\243\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
